# Sudoku Solver

To use the Sudoku Solver, first download and open this ipynb file or open it online [here](https://mybinder.org/v2/gh/chartung17/sudoku-solver/main?filepath=SudokuSolver.ipynb), then run the following cell and follow the prompts to input a puzzle.

In [1]:
# get puzzle as user input
import re
print('Please enter a Sudoku puzzle as a string, one row at a time, with a 0 to represent each blank space.')
print('Each row must include exactly 9 numeric characters and any number of non-numeric characters.')
sudoku_str = ''
for row in range(9):
    next_line = re.sub('[^0-9]', '', input(f'Enter row {row+1}: '))
    if len(next_line) != 9:
        raise ValueError('Each row must contain exactly 9 numeric characters')
    sudoku_str += next_line
print('')
print('')

# convert input to matrix
sudoku = [[[0,0,0] for j in range(3)] for i in range(9)]
for char in range(81):
    row = char // 9
    col = char % 9
    sudoku[row][col // 3][col % 3] = int(sudoku_str[char])

# start timer
import time
start = time.perf_counter()

# define class for unsolved squares
class Unsolved:
    def __init__(self, i, j, k, poss):
        # i,j,k indicates position of the square
        self.i = i
        self.j = j
        self.k = k
        # poss[0] indicates number of possibilities for the square
        # poss[1]-poss[9] indicate whether square could have the given value
        self.poss = poss
        
import copy
# function to print puzzle
def print_puzzle():
    # copy puzzle and replace unsolved squares with 0 in the copy
    sudoku_copy = copy.deepcopy(sudoku)
    for i in range(9):
        for j in range(3):
            for k in range(3):
                if isinstance(sudoku[i][j][k], Unsolved):
                    sudoku_copy[i][j][k] = 0
    # print the puzzle one row at a time with a blank line after every three rows
    for i in range(3):
        print(sudoku_copy[i])
    print('')
    for i in range(3):
        print(sudoku_copy[i+3])
    print('')
    for i in range(3):
        print(sudoku_copy[i+6])
    print('')
    del(sudoku_copy)

# print original puzzle
print('Original puzzle:')
print_puzzle()
print('')

# create lists to indicate what has been solved already in each row, column, and box
# within these lists, each nested list indicates a particular row, column, or box in the puzzle
# the element in position 0 of a nested list counts the number of solved positions within the row/column/box
# the elements is positions 1-9 of a nested list indicate whether the corresponding number has been placed yet
rows = [[0 for col in range(10)] for row in range(9)]
cols = [[0 for col in range(10)] for row in range(9)]
boxes = [[0 for col in range(10)] for row in range(9)]

# function to indicate whether the specified value was already used in the same
# row, column, or box as the given unsolved square
def invalid(unsolved, val):
    i = unsolved.i
    j = unsolved.j
    k = unsolved.k
    # check row
    if rows[i][val]:
        return True
    # check col
    if cols[3*j+k][val]:
        return True
    # check box
    if boxes[3*(i // 3) + j][val]:
        return True
    return False

# function to update rows, cols, and boxes when a square is filled in
def update_square(i, j, k, val):
    rows[i][val] = 1
    rows[i][0] += 1
    cols[3*j + k][val] = 1
    cols[3*j + k][0] += 1
    boxes[3*(i // 3) + j][val] = 1
    boxes[3*(i // 3) + j][0] += 1

# fill in rows, cols, and boxes based on original puzzle
for i in range(9):
    for j in range(3):
        for k in range(3):
            val = sudoku[i][j][k]
            if val:
                update_square(i, j, k, val)
                
# create objects for all unsolved squares
count_unsolved = 0
for i in range(9):
    for j in range(3):
        for k in range(3):
            if not sudoku[i][j][k]:
                # determine which values are possible based on the rows, cols, and boxes lists
                poss = [0 for row in range(10)]
                for x in range(1,10):
                    if (not rows[i][x]) and (not cols[3*j + k][x]) and (not boxes[3*(i // 3) + j][x]):
                        poss[x] = 1
                        poss[0] += 1
                # if there is only one possibility, fill in the square, otherwise create an Unsolved object
                if poss[0] == 1:
                    for x in range(1,10):
                        if poss[x]:
                            sudoku[i][j][k] = x
                            update_square(i, j, k, x)
                            break
                else:
                    sudoku[i][j][k] = Unsolved(i, j, k, poss)
                    count_unsolved += 1
                    
# function to update a given unsolved square
# returns 0 if square is still unsolved and 1 if it is solved
def update_unsolved(val):
    i = val.i
    j = val.j
    k = val.k
    # update the square's poss list based on which digits have already been used in the same row, col, or box
    for x in range(1,10):
        if val.poss[x]:
            if rows[i][x] or cols[3*j + k][x] or boxes[3*(i // 3) + j][x]:
                val.poss[x] = 0
                val.poss[0] -= 1
    # if only one possible value is left, fill in the square are return 1
    if val.poss[0] == 1:
        for x in range(1,10):
            if val.poss[x]:
                sudoku[i][j][k] = x
                update_square(i, j, k, x)
                del val
                return 1
    # if the square has not been filled in, return 0
    else:
        return 0

# function to find and return all unsolved squares in a given row, col, or box
def find_unsolved(dim, num):
    result = []
    # dim must be 'row', 'col', or 'box'
    if dim == 'row':
        for j in range(3):
            for k in range(3):
                val = sudoku[num][j][k]
                if isinstance(val, Unsolved):
                    result.append(val)
    elif dim == 'col':
        j = num // 3
        k = num % 3
        for i in range(9):
            val = sudoku[i][j][k]
            if isinstance(val, Unsolved):
                result.append(val)
    elif dim == 'box':
        x = num // 3
        j = num % 3
        for i in range(3*x, 3*x+3):
            for k in range(3):
                val = sudoku[i][j][k]
                if isinstance(val, Unsolved):
                    result.append(val)
    return result

# function to check for numbers that only have one possible position in a given row, col, or box
# returns the number of squares that were solved during the execution of the function
def check_unique(list_unsolved, dim, num):
    count = 0
    # iterate through all digits
    for x in range(1,10):
        loc = 0
        # iterate through all unsolved squares
        for unsolved in list_unsolved:
            # check if the current square could contain the current digit
            if unsolved.poss[x]:
                if invalid(unsolved, x):
                    unsolved.poss[x] = 0
                    unsolved.poss[0] -= 1
                # if this is the first square that could contain the current digit, mark it as a possible location
                elif loc == 0:
                    loc = unsolved
                # if multiple possible locations are found, move on to the next digit
                else:
                    loc = 0
                    break
        # if exactly one possible location was found, fill in that square and increment the count
        if loc != 0:
            i = loc.i
            j = loc.j
            k = loc.k
            sudoku[i][j][k] = x
            update_square(i, j, k, x)
            list_unsolved.remove(loc)
            del loc
            count += 1
    # return the number of squares that were solved within this function call
    return count

# function to check for pairs that must go in a certain pair of boxes
# returns True if at least one pair is found and false otherwise
def check_pairs():
    progress_made = False
    # look for each possible pair of two numbers between 1 and 9
    for x in range(2,10):
        for y in range(1,x):
            # check for pairs in each row
            for i in range(9):
                poss = []
                for j in range(3):
                    for k in range(3):
                        val = sudoku[i][j][k]
                        if isinstance(val, Unsolved) and (val.poss[x] or val.poss[y]):
                            poss.append(val)
                        elif (val == x) or (val == y):
                            poss = [0,0,0,0]
                if len(poss) == 2:
                    # if pair found, neither square can have any other value
                    for z in range(1,10):
                        if poss[0].poss[z] and (z != x) and (z != y):
                            progress_made = True
                            poss[0].poss[z] = 0
                            poss[0].poss[0] -= 1
                        if poss[1].poss[z] and (z != x) and (z != y):
                            progress_made = True
                            poss[1].poss[z] = 0
                            poss[1].poss[0] -= 1
                    # if pair are both in same box, no other square in that box can have either paired value
                    if poss[0].j == poss[1].j:
                        j = poss[0].j
                        for i1 in range(3*(i // 3), 3*(i //3)+3):
                            if i != i1:
                                for k in range(3):
                                    val = sudoku[i1][j][k]
                                    if isinstance(val, Unsolved):
                                        if val.poss[x]:
                                            val.poss[x] = 0
                                            val.poss[0] -= 1
                                            progress_made = True
                                        if val.poss[y]:
                                            val.poss[y] = 0
                                            val.poss[0] -= 1
                                            progress_made = True
            # check for pairs in each col
            for j in range(3):
                for k in range(3):
                    poss = []
                    for i in range(9):
                        val = sudoku[i][j][k]
                        if isinstance(val, Unsolved) and (val.poss[x] or val.poss[y]):
                            poss.append(val)
                        elif (val == x) or (val == y):
                            poss = [0,0,0,0]
                    if len(poss) == 2:
                        # if pair found, neither square can have any other value
                        for z in range(1,10):
                            if poss[0].poss[z] and (z != x) and (z != y):
                                progress_made = True
                                poss[0].poss[z] = 0
                                poss[0].poss[0] -= 1
                            if poss[1].poss[z] and (z != x) and (z != y):
                                progress_made = True
                                poss[1].poss[z] = 0
                                poss[1].poss[0] -= 1
                        # if pair are both in same box, no other square in that box can have either paired value
                        if (poss[0].i // 3) == (poss[1].i // 3):
                            i = poss[1].i
                            for i1 in range(3*(i // 3), 3*(i //3)+3):
                                for k1 in range(3):
                                    if k != k1:
                                        val = sudoku[i1][j][k1]
                                        if isinstance(val, Unsolved):
                                            if val.poss[x]:
                                                val.poss[x] = 0
                                                val.poss[0] -= 1
                                                progress_made = True
                                            if val.poss[y]:
                                                val.poss[y] = 0
                                                val.poss[0] -= 1
                                                progress_made = True
            # check for pairs in each box
            for i1 in range(3):
                for j in range(3):
                    poss = []
                    for i2 in range(3):
                        for k in range(3):
                            val = sudoku[3*i1 + i2][j][k]
                            if isinstance(val, Unsolved) and (val.poss[x] or val.poss[y]):
                                poss.append(val)
                            elif (val == x) or (val == y):
                                poss = [0,0,0,0]
                    if len(poss) == 2:
                        # if pair found, neither square can have any other value
                        for z in range(1,10):
                            if poss[0].poss[z] and (z != x) and (z != y):
                                progress_made = True
                                poss[0].poss[z] = 0
                                poss[0].poss[0] -= 1
                            if poss[1].poss[z] and (z != x) and (z != y):
                                progress_made = True
                                poss[1].poss[z] = 0
                                poss[1].poss[0] -= 1
    return progress_made



# function to check for triples that must go in a certain triple of boxes
# returns True if at least one triple is found and false otherwise
def check_triples():
    progress_made = False
    # look for each possible triple of three numbers between 1 and 9
    for x in range(3,10):
        for y in range(2,x):
            for z in range(1,y):
                # check for triples in each row
                for i in range(9):
                    poss = []
                    for j in range(3):
                        for k in range(3):
                            val = sudoku[i][j][k]
                            if isinstance(val, Unsolved) and (val.poss[x] or val.poss[y] or val.poss[z]):
                                poss.append(val)
                            elif (val == x) or (val == y) or (val == z):
                                poss = [0,0,0,0]
                    if len(poss) == 3:
                        # if triple found, none of the three squares can have any other value
                        for q in range(1,10):
                            for w in range(3):
                                if poss[w].poss[q] and (q != x) and (q != y) and (q != z):
                                    progress_made = True
                                    poss[w].poss[q] = 0
                                    poss[w].poss[0] -= 1
                        # if triple are all in same box, no other square in that box can have any of the three values
                        if poss[0].j == poss[1].j == poss[2].j:
                            j = poss[0].j
                            for i1 in range(3*(i // 3), 3*(i //3)+3):
                                if i != i1:
                                    for k in range(3):
                                        val = sudoku[i1][j][k]
                                        if isinstance(val, Unsolved):
                                            if val.poss[x]:
                                                val.poss[x] = 0
                                                val.poss[0] -= 1
                                                progress_made = True
                                            if val.poss[y]:
                                                val.poss[y] = 0
                                                val.poss[0] -= 1
                                                progress_made = True
                # check for triples in each col
                for j in range(3):
                    for k in range(3):
                        poss = []
                        for i in range(9):
                            val = sudoku[i][j][k]
                            if isinstance(val, Unsolved) and (val.poss[x] or val.poss[y] or val.poss[z]):
                                poss.append(val)
                            elif (val == x) or (val == y) or (val == z):
                                poss = [0,0,0,0]
                        if len(poss) == 3:
                            # if triple found, none of the three squares can have any other value
                            for q in range(1,10):
                                for w in range(3):
                                    if poss[w].poss[q] and (q != x) and (q != y) and (q != z):
                                        progress_made = True
                                        poss[w].poss[q] = 0
                                        poss[w].poss[0] -= 1
                            # if triple are all in same box, no other square in that box can have any of the three values
                            if (poss[0].i // 3) == (poss[1].i // 3) == (poss[2].i // 3):
                                i = poss[1].i
                                for i1 in range(3*(i // 3), 3*(i //3)+3):
                                    for k1 in range(3):
                                        if k != k1:
                                            val = sudoku[i1][j][k1]
                                            if isinstance(val, Unsolved):
                                                if val.poss[x]:
                                                    val.poss[x] = 0
                                                    val.poss[0] -= 1
                                                    progress_made = True
                                                if val.poss[y]:
                                                    val.poss[y] = 0
                                                    val.poss[0] -= 1
                                                    progress_made = True
                # check for triples in each box
                for i1 in range(3):
                    for j in range(3):
                        poss = []
                        for i2 in range(3):
                            for k in range(3):
                                val = sudoku[3*i1 + i2][j][k]
                                if isinstance(val, Unsolved) and (val.poss[x] or val.poss[y] or val.poss[z]):
                                    poss.append(val)
                                elif (val == x) or (val == y) or (val == z):
                                    poss = [0,0,0,0]
                        if len(poss) == 3:
                            # if triple found, none of the three squares can have any other value
                            for q in range(1,10):
                                for w in range(3):
                                    if poss[w].poss[q] and (q != x) and (q != y) and (q != z):
                                        progress_made = True
                                        poss[w].poss[q] = 0
                                        poss[w].poss[0] -= 1
    return progress_made                        

# solve the puzzle
while count_unsolved:
    prev_count = 0
    while count_unsolved and (count_unsolved != prev_count):
        prev_count = count_unsolved
        # update possibilities for all unsolved squares
        for i in range(9):
            for j in range(3):
                for k in range(3):
                    val = sudoku[i][j][k]
                    if isinstance(val, Unsolved):
                        count_unsolved -= update_unsolved(val)
        # check for numbers which only have one possible position in any row, col, or box
        for num in range(9):
            count_unsolved -= check_unique(find_unsolved('row', num), 'row', num)
            count_unsolved -= check_unique(find_unsolved('col', num), 'col', num)
            count_unsolved -= check_unique(find_unsolved('box', num), 'box', num)
    if not check_pairs():
        if not check_triples():
            break

# print solution and stop timer
if count_unsolved:
    print('Unable to solve puzzle. Partial solution:')
else:
    print('Solution:')
print_puzzle()
stop = time.perf_counter()
print(f'Execution time: {stop - start:0.4f} seconds')

Please enter a Sudoku puzzle as a string, one row at a time, with a 0 to represent each blank space.
Each row must include exactly 9 numeric characters and any number of non-numeric characters.
Enter row 1: 008 000 073
Enter row 2: 250 400 000
Enter row 3: 000 030 090
Enter row 4: 802 000 000
Enter row 5: 014 060 320
Enter row 6: 000 000 907
Enter row 7: 040 020 000
Enter row 8: 000 009 051
Enter row 9: 380 000 700


Original puzzle:
[[0, 0, 8], [0, 0, 0], [0, 7, 3]]
[[2, 5, 0], [4, 0, 0], [0, 0, 0]]
[[0, 0, 0], [0, 3, 0], [0, 9, 0]]

[[8, 0, 2], [0, 0, 0], [0, 0, 0]]
[[0, 1, 4], [0, 6, 0], [3, 2, 0]]
[[0, 0, 0], [0, 0, 0], [9, 0, 7]]

[[0, 4, 0], [0, 2, 0], [0, 0, 0]]
[[0, 0, 0], [0, 0, 9], [0, 5, 1]]
[[3, 8, 0], [0, 0, 0], [7, 0, 0]]


Solution:
[[9, 6, 8], [5, 1, 2], [4, 7, 3]]
[[2, 5, 3], [4, 9, 7], [1, 8, 6]]
[[4, 7, 1], [8, 3, 6], [2, 9, 5]]

[[8, 9, 2], [1, 7, 3], [5, 6, 4]]
[[7, 1, 4], [9, 6, 5], [3, 2, 8]]
[[5, 3, 6], [2, 8, 4], [9, 1, 7]]

[[1, 4, 5], [7, 2, 8], [6, 3, 9]]
[[

In [2]:
# if the Sudoku Solver is unable to solve the puzzle, run this cell for details on possible options for each unsolved square
# rows and columns in output use 1-indexing
count = 0
for i in range(9):
    for j in range(3):
        for k in range(3):
            val = sudoku[i][j][k]
            if isinstance(val, Unsolved):
                row = i+1
                col = 3*j + k + 1
                poss = []
                for x in range(1,10):
                    if val.poss[x]:
                        poss.append(x)
                print(f'Possibilities for row {row}, column {col}:')
                print(val.poss)
                print('')
                